# Production-Grade Playwright Test Generation System

This notebook implements a clean, production-ready system for automated generation of Playwright Page Object Model classes using AI agents.

## Architecture
- **Dependency Injection**: Service management and configuration
- **Command Pattern**: Different execution strategies
- **Repository Pattern**: Data access abstraction
- **Clean Separation**: Single responsibility and SOLID principles

In [12]:
# Install required dependencies
%pip install openai semantic-kernel pydantic python-dotenv

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [13]:
"""Core imports and foundational setup."""

import asyncio
import logging
import os
import yaml
import time
from abc import ABC, abstractmethod
from contextlib import asynccontextmanager
from dataclasses import dataclass, field
from datetime import datetime, timezone
from enum import Enum
from pathlib import Path
from typing import Any, Dict, List, Optional, Callable, Awaitable, TypeVar
from uuid import uuid4, UUID

from dotenv import load_dotenv
from pydantic import BaseModel, Field, validator

from semantic_kernel import Kernel
from semantic_kernel.functions import KernelArguments
from semantic_kernel.connectors.ai.open_ai import OpenAIChatCompletion
from semantic_kernel.agents import ChatCompletionAgent, ChatHistoryAgentThread
from semantic_kernel.prompt_template import PromptTemplateConfig

# Load environment variables
load_dotenv()

# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger(__name__)

# Type aliases
T = TypeVar('T')
ServiceFactory = Callable[[], Awaitable[T]]

## 🏗️ Core Domain Models

This section defines the fundamental data structures and business objects:

### **Enums**
- `ExecutionMode`: Defines how the system operates (single file vs batch processing)
- `GenerationStatus`: Tracks the lifecycle of code generation operations

### **Value Objects**
- `ExecutionContext`: Immutable context containing session information and metadata
- `PageDefinition`: Represents a page description with validation rules
- `GenerationResult`: Comprehensive result tracking with success/failure states

### **Key Features**
- **Pydantic Validation**: Automatic data validation and serialization
- **Type Safety**: Complete type hints for all properties
- **Immutability**: Where appropriate, using frozen dataclasses
- **Rich Metadata**: Timestamps, UUIDs, and tracking information

In [14]:
"""Core domain models and enums."""

class ExecutionMode(Enum):
    """Execution modes for the test generation system."""
    SINGLE_FILE = "single_file"
    DIRECTORY_BATCH = "directory_batch"


class GenerationStatus(Enum):
    """Status enumeration for generation operations."""
    PENDING = "pending"
    IN_PROGRESS = "in_progress"
    COMPLETED = "completed"
    FAILED = "failed"


@dataclass(frozen=True)
class ExecutionContext:
    """Immutable execution context for operations."""
    session_id: UUID = field(default_factory=uuid4)
    timestamp: datetime = field(default_factory=lambda: datetime.now(timezone.utc))
    mode: ExecutionMode = ExecutionMode.SINGLE_FILE
    metadata: Dict[str, Any] = field(default_factory=dict)


class PageDefinition(BaseModel):
    """Page definition with validation."""
    
    id: UUID = Field(default_factory=uuid4)
    name: str = Field(..., min_length=1, max_length=100)
    content: str = Field(..., min_length=10)
    file_path: Optional[Path] = None
    created_at: datetime = Field(default_factory=lambda: datetime.now(timezone.utc))
    
    class Config:
        arbitrary_types_allowed = True
        json_encoders = {
            Path: str,
            datetime: lambda v: v.isoformat(),
            UUID: str
        }
    
    @validator('name')
    def validate_name(cls, v):
        """Validate page name format."""
        if not v.replace('_', '').replace('-', '').isalnum():
            raise ValueError('Page name must contain only alphanumeric characters, hyphens, and underscores')
        return v


class GenerationResult(BaseModel):
    """Generation result with tracking information."""
    
    id: UUID = Field(default_factory=uuid4)
    page_definition_id: UUID
    page_name: str
    generated_code: str = ""
    status: GenerationStatus
    error_message: Optional[str] = None
    error_type: Optional[str] = None
    execution_time: Optional[float] = None
    timestamp: datetime = Field(default_factory=lambda: datetime.now(timezone.utc))
    context: Optional[ExecutionContext] = None
    
    class Config:
        json_encoders = {
            datetime: lambda v: v.isoformat(),
            UUID: str
        }
    
    @property
    def is_successful(self) -> bool:
        """Check if generation was successful."""
        return self.status == GenerationStatus.COMPLETED
    
    def mark_as_failed(self, error: Exception, execution_time: Optional[float] = None) -> None:
        """Mark result as failed with error details."""
        self.status = GenerationStatus.FAILED
        self.error_message = str(error)
        self.error_type = type(error).__name__
        if execution_time:
            self.execution_time = execution_time
    
    def mark_as_completed(self, code: str, execution_time: float) -> None:
        """Mark result as completed with generated code."""
        self.status = GenerationStatus.COMPLETED
        self.generated_code = code
        self.execution_time = execution_time

C:\Users\montr\AppData\Local\Temp\ipykernel_12412\3846900277.py:43: PydanticDeprecatedSince20: Pydantic V1 style `@validator` validators are deprecated. You should migrate to Pydantic V2 style `@field_validator` validators, see the migration guide for more details. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  @validator('name')


## ⚙️ Configuration Management

Production-grade configuration system with validation and multiple sources:

### **ApplicationConfig**
- **API Settings**: OpenAI API key and model configuration
- **Performance**: Timeout and retry settings
- **Validation**: Automatic validation of configuration values

### **ConfigurationManager**
- **Environment Variables**: Automatic loading from `OPENAI_API_KEY`
- **Override Support**: Runtime configuration overrides
- **Validation**: Ensures all required settings are present and valid

### **Key Features**
- **Type Safety**: Pydantic models ensure configuration correctness
- **Environment Integration**: Seamless `.env` file support
- **Validation**: API key format validation and required field checks
- **Flexibility**: Easy to extend with new configuration options

In [15]:
"""Configuration management."""

class ApplicationConfig(BaseModel):
    """Application configuration."""
    
    # API Configuration
    openai_api_key: str = Field(..., min_length=20)
    chat_model_id: str = Field(default="gpt-3.5-turbo")
    
    # Service Configuration
    chat_service_id: str = Field(default="chat-gpt")
    
    # Performance Configuration
    default_timeout_ms: int = Field(default=30000, ge=1000, le=300000)
    max_retries: int = Field(default=3, ge=1, le=10)
    
    @validator('openai_api_key')
    def validate_api_key(cls, v):
        """Validate OpenAI API key format."""
        if not v.startswith('sk-'):
            raise ValueError('Invalid OpenAI API key format')
        return v


class ConfigurationManager:
    """Manages application configuration."""
    
    def __init__(self) -> None:
        self._config: Optional[ApplicationConfig] = None
    
    def load_configuration(self, overrides: Optional[Dict[str, Any]] = None) -> ApplicationConfig:
        """Load configuration from environment and overrides."""
        config_data = {}
        
        # Load from environment variables
        if os.getenv('OPENAI_API_KEY'):
            config_data['openai_api_key'] = os.getenv('OPENAI_API_KEY')
        
        # Apply overrides
        if overrides:
            config_data.update(overrides)
        
        try:
            self._config = ApplicationConfig(**config_data)
            logger.info("Configuration loaded successfully")
            return self._config
        except Exception as e:
            logger.error(f"Configuration validation failed: {e}")
            raise
    
    def get_config(self) -> ApplicationConfig:
        """Get the current configuration."""
        if not self._config:
            raise RuntimeError("Configuration not loaded. Call load_configuration() first.")
        return self._config

C:\Users\montr\AppData\Local\Temp\ipykernel_12412\1356973872.py:17: PydanticDeprecatedSince20: Pydantic V1 style `@validator` validators are deprecated. You should migrate to Pydantic V2 style `@field_validator` validators, see the migration guide for more details. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  @validator('openai_api_key')


## 🔄 Dependency Injection Container

Lightweight dependency injection system for service management:

### **ServiceContainer**
- **Instance Registration**: Register pre-created service instances
- **Factory Registration**: Register factory functions for lazy initialization
- **Service Resolution**: Automatic service creation and caching
- **Lifecycle Management**: Proper disposal of services and resources

### **Key Features**
- **Simple API**: Easy registration and resolution patterns
- **Async Support**: Full support for async service factories
- **Error Handling**: Clear error messages for missing services
- **Resource Cleanup**: Automatic disposal of services with cleanup methods

This enables clean separation of concerns and makes the system highly testable by allowing easy service mocking and replacement.

In [16]:
"""Simple dependency injection container."""

class ServiceContainer:
    """Simple dependency injection container."""
    
    def __init__(self) -> None:
        self._services: Dict[type, Any] = {}
        self._factories: Dict[type, ServiceFactory] = {}
    
    def register_instance(self, service_type: type, instance: Any) -> 'ServiceContainer':
        """Register a specific instance."""
        self._services[service_type] = instance
        return self
    
    def register_factory(self, service_type: type, factory: ServiceFactory) -> 'ServiceContainer':
        """Register a service factory."""
        self._factories[service_type] = factory
        return self
    
    async def get_service(self, service_type: type) -> Any:
        """Get an instance of the requested service."""
        # Return existing instance
        if service_type in self._services:
            return self._services[service_type]
        
        # Create from factory
        if service_type in self._factories:
            instance = await self._factories[service_type]()
            self._services[service_type] = instance
            return instance
        
        raise ValueError(f"Service {service_type.__name__} is not registered")
    
    async def dispose(self) -> None:
        """Dispose of all services."""
        for instance in self._services.values():
            if hasattr(instance, 'dispose'):
                try:
                    if asyncio.iscoroutinefunction(instance.dispose):
                        await instance.dispose()
                    else:
                        instance.dispose()
                except Exception as e:
                    logger.warning(f"Error disposing service: {e}")
        
        self._services.clear()
        self._factories.clear()

## 📚 Repository Pattern

Data access layer using the Repository pattern for clean data management:

### **Abstract Interfaces**
- `IPageDefinitionRepository`: Contract for page definition storage and retrieval
- `IGenerationResultRepository`: Contract for result tracking and session management

### **In-Memory Implementations**
- `InMemoryPageDefinitionRepository`: Simple in-memory storage for page definitions
- `InMemoryGenerationResultRepository`: Session-based result tracking

### **Key Benefits**
- **Abstraction**: Clean separation between business logic and data storage
- **Testability**: Easy to mock for unit testing
- **Flexibility**: Can be replaced with database, file system, or cloud storage
- **Session Management**: Track results by execution session for better organization

The repository pattern makes it easy to swap storage implementations without changing business logic.

In [17]:
"""Repository pattern for data access."""

class IPageDefinitionRepository(ABC):
    """Abstract repository for page definitions."""
    
    @abstractmethod
    async def save(self, page_definition: PageDefinition) -> None:
        """Save a page definition."""
        pass
    
    @abstractmethod
    async def get_all(self) -> List[PageDefinition]:
        """Get all page definitions."""
        pass


class IGenerationResultRepository(ABC):
    """Abstract repository for generation results."""
    
    @abstractmethod
    async def save_result(self, result: GenerationResult) -> None:
        """Save a generation result."""
        pass
    
    @abstractmethod
    async def get_results_by_session(self, session_id: UUID) -> List[GenerationResult]:
        """Get results by session ID."""
        pass


class InMemoryPageDefinitionRepository(IPageDefinitionRepository):
    """In-memory implementation of page definition repository."""
    
    def __init__(self) -> None:
        self._pages: Dict[UUID, PageDefinition] = {}
    
    async def save(self, page_definition: PageDefinition) -> None:
        """Save a page definition."""
        self._pages[page_definition.id] = page_definition
        logger.debug(f"Saved page definition: {page_definition.name}")
    
    async def get_all(self) -> List[PageDefinition]:
        """Get all page definitions."""
        return list(self._pages.values())


class InMemoryGenerationResultRepository(IGenerationResultRepository):
    """In-memory implementation of generation result repository."""
    
    def __init__(self) -> None:
        self._results: List[GenerationResult] = []
    
    async def save_result(self, result: GenerationResult) -> None:
        """Save generation result to memory."""
        self._results.append(result)
        logger.debug(f"Saved generation result for: {result.page_name}")
    
    async def get_results_by_session(self, session_id: UUID) -> List[GenerationResult]:
        """Get results by session ID."""
        return [
            result for result in self._results 
            if result.context and result.context.session_id == session_id
        ]

## 🤖 AI Service Layer

Production-grade AI integration for Playwright code generation:

### **Service Architecture**
- `IAIService`: Abstract interface for AI code generation
- `BasePageTemplateService`: Manages the base page template with configuration
- `PromptTemplateService`: Handles AI prompt template configuration
- `SemanticKernelAIService`: OpenAI integration using Microsoft Semantic Kernel

### **Key Features**
- **Retry Logic**: Exponential backoff for resilient API calls
- **Template Management**: Dynamic base page template generation
- **Code Cleaning**: Automatic removal of markdown fencing and formatting
- **Error Handling**: Comprehensive error tracking and reporting
- **Async Operations**: Non-blocking AI service calls

### **Prompt Engineering**
- **Structured Prompts**: Detailed instructions for high-quality code generation
- **Best Practices**: Guidance for async/await, type hints, and data-testid selectors
- **Examples**: Clear input/output examples for consistent results

The AI service layer abstracts OpenAI integration and can be easily extended with other AI providers.

In [18]:
"""AI service layer for code generation."""

class IAIService(ABC):
    """Abstract interface for AI services."""
    
    @abstractmethod
    async def generate_code(self, page_definition: PageDefinition, context: ExecutionContext) -> GenerationResult:
        """Generate code for a page definition."""
        pass


class BasePageTemplateService:
    """Service for managing base page templates."""
    
    def __init__(self, config: ApplicationConfig) -> None:
        self.config = config
    
    def get_base_template(self) -> str:
        """Get the base page template."""
        return f'''
from typing import Optional
from playwright.async_api import Page, Locator


class BasePage:
    """
    Base page class with common functionality for all page objects.
    
    This class provides shared methods and utilities for all page objects,
    following clean architecture principles.
    """
    
    def __init__(self, page: Page) -> None:
        """
        Initialize the base page with a Playwright page instance.
        
        Args:
            page: The Playwright page object for browser interactions
        """
        self.page = page
        self.default_timeout = {self.config.default_timeout_ms}
    
    async def navigate(self, url: str, wait_until: str = "domcontentloaded") -> None:
        """
        Navigate to the specified URL.
        
        Args:
            url: The target URL to navigate to
            wait_until: When to consider navigation complete
        """
        await self.page.goto(url, wait_until=wait_until, timeout=self.default_timeout)
    
    async def wait_for_element(
        self, 
        selector: str, 
        timeout: Optional[int] = None,
        state: str = "visible"
    ) -> Locator:
        """
        Wait for an element to reach the specified state.
        
        Args:
            selector: CSS selector or data-testid locator
            timeout: Maximum time to wait in milliseconds
            state: Element state to wait for
            
        Returns:
            The located element
        """
        timeout = timeout or self.default_timeout
        locator = self.page.locator(selector)
        await locator.wait_for(state=state, timeout=timeout)
        return locator
    
    async def get_title(self) -> str:
        """
        Get the current page title.
        
        Returns:
            The page title as a string
        """
        return await self.page.title()
'''


class PromptTemplateService:
    """Service for managing AI prompt templates."""
    
    def get_template_config(self) -> PromptTemplateConfig:
        """Get the prompt template configuration."""
        template_yaml = '''
name: GeneratePlaywrightPageObject
template: |
  ## Role
  You are an expert Python developer specializing in Playwright test automation.

  ## Instructions
  Generate a high-quality Playwright page object model that inherits from BasePage.
  
  ### Requirements
  1. Use async/await for all Playwright operations
  2. Include comprehensive type hints and docstrings
  3. Use descriptive, self-documenting method and variable names
  4. Prefer data-testid selectors: [data-testid="element-name"]
  5. Handle wait conditions appropriately
  6. Follow clean code practices

  ### Response Format
  Respond only with clean Python code without markdown fencing.

  ## Context
  All page objects inherit from this BasePage:
  {{$base_page_code}}

  ## Example
  ### Input
  ```
  # Login Page
  ## Tasks
  ### Login
  Authenticate user with credentials
  #### Steps
  1. Enter username
  2. Enter password
  3. Click login button
  ```

  ### Output
  ```python
  class LoginPage(BasePage):
      """Page object for user authentication functionality."""
      
      def __init__(self, page: Page) -> None:
          """Initialize login page with required elements."""
          super().__init__(page)
          self.username_input = page.locator('[data-testid="username-input"]')
          self.password_input = page.locator('[data-testid="password-input"]')
          self.login_button = page.locator('[data-testid="login-button"]')

      async def login(self, username: str, password: str) -> None:
          """Authenticate user with provided credentials."""
          await self.username_input.fill(username)
          await self.password_input.fill(password)
          await self.login_button.click()
  ```

  ---
  ### Input
  ```
  {{$page_description}}
  ```

template_format: semantic-kernel
description: Generates Playwright page object models
input_variables:
    - name: base_page_code
      description: The base page template code
      is_required: true
    - name: page_description
      description: The page description for code generation
      is_required: true
'''
        
        try:
            data = yaml.safe_load(template_yaml)
            return PromptTemplateConfig(**data)
        except Exception as e:
            logger.error(f"Failed to build prompt template: {e}")
            raise


class SemanticKernelAIService(IAIService):
    """AI service using Semantic Kernel for code generation."""
    
    def __init__(
        self, 
        config: ApplicationConfig,
        template_service: BasePageTemplateService,
        prompt_service: PromptTemplateService
    ) -> None:
        self.config = config
        self.template_service = template_service
        self.prompt_service = prompt_service
        self.kernel: Optional[Kernel] = None
        self.agent: Optional[ChatCompletionAgent] = None
        self._initialized = False
    
    async def initialize(self) -> None:
        """Initialize the AI service."""
        if self._initialized:
            return
        
        try:
            self.kernel = Kernel()
            
            # Configure chat completion service
            chat_service = OpenAIChatCompletion(
                service_id=self.config.chat_service_id,
                ai_model_id=self.config.chat_model_id,
                api_key=self.config.openai_api_key
            )
            
            self.kernel.add_service(chat_service)
            
            # Create chat completion agent
            self.agent = ChatCompletionAgent(
                kernel=self.kernel,
                service=chat_service,
                prompt_template_config=self.prompt_service.get_template_config(),
                name="PlaywrightCodeGenerator",
                instructions=(
                    "You are a senior Python developer specializing in test automation. "
                    "Generate clean, production-quality Playwright page objects."
                )
            )
            
            self._initialized = True
            logger.info("AI service initialized successfully")
            
        except Exception as e:
            logger.error(f"Failed to initialize AI service: {e}")
            raise
    
    async def generate_code(self, page_definition: PageDefinition, context: ExecutionContext) -> GenerationResult:
        """Generate code for a page definition."""
        if not self._initialized:
            await self.initialize()
        
        result = GenerationResult(
            page_definition_id=page_definition.id,
            page_name=page_definition.name,
            status=GenerationStatus.IN_PROGRESS,
            context=context
        )
        
        start_time = time.time()
        
        try:
            # Prepare generation arguments
            arguments = KernelArguments(
                page_description=page_definition.content,
                base_page_code=self.template_service.get_base_template()
            )
            
            # Generate code with retry logic
            generated_code = await self._generate_with_retry(arguments)
            
            # Clean the generated code
            cleaned_code = self._clean_generated_code(generated_code)
            
            execution_time = time.time() - start_time
            result.mark_as_completed(cleaned_code, execution_time)
            
            logger.info(f"Successfully generated code for {page_definition.name} in {execution_time:.2f}s")
            
        except Exception as e:
            execution_time = time.time() - start_time
            result.mark_as_failed(e, execution_time)
            logger.error(f"Code generation failed for {page_definition.name}: {e}")
        
        return result
    
    async def _generate_with_retry(self, arguments: KernelArguments) -> str:
        """Generate code with retry logic."""
        thread: Optional[ChatHistoryAgentThread] = None
        
        for attempt in range(self.config.max_retries):
            try:
                response = await self.agent.get_response(
                    thread=thread,
                    arguments=arguments
                )
                
                if response and response.content and response.content.content:
                    return response.content.content
                
                raise ValueError("Empty response from AI service")
                
            except Exception as e:
                if attempt == self.config.max_retries - 1:
                    raise
                
                delay = min(2 ** attempt, 10)  # Exponential backoff
                logger.warning(f"Generation attempt {attempt + 1} failed: {e}. Retrying in {delay}s...")
                await asyncio.sleep(delay)
        
        # Cleanup
        if thread:
            try:
                await thread.delete()
            except Exception:
                pass
        
        raise RuntimeError(f"Code generation failed after {self.config.max_retries} attempts")
    
    def _clean_generated_code(self, code: str) -> str:
        """Clean and format generated code."""
        # Remove markdown fencing
        cleaned = code.replace('```python', '').replace('```', '')
        
        # Normalize whitespace and line endings
        lines = [line.rstrip() for line in cleaned.splitlines()]
        
        # Remove empty lines at start and end
        while lines and not lines[0].strip():
            lines.pop(0)
        while lines and not lines[-1].strip():
            lines.pop()
        
        return '\n'.join(lines)

## ⚡ Command Pattern Implementation

Flexible execution strategies using the Command pattern:

### **Command Architecture**
- `ICommand`: Abstract interface defining the command contract
- `BaseCommand`: Common functionality for all command implementations
- `SingleFileGenerationCommand`: Processes individual page definitions
- `BatchGenerationCommand`: Handles multiple pages with progress tracking
- `CommandFactory`: Creates appropriate commands based on execution mode

### **Execution Modes**
- **Single File**: Process one page definition with detailed output
- **Batch Processing**: Handle multiple pages efficiently with progress updates
- **Error Recovery**: Continue processing even if individual pages fail

### **Key Benefits**
- **Flexibility**: Easy to add new execution strategies
- **Separation of Concerns**: Business logic separated from execution strategy
- **Error Isolation**: Failures in one page don't stop batch processing
- **Progress Tracking**: Real-time feedback on long-running operations

The command pattern makes the system extensible and allows different execution strategies without changing core logic.

In [19]:
"""Command pattern for different execution strategies."""

class ICommand(ABC):
    """Abstract command interface."""
    
    @abstractmethod
    async def execute(self) -> List[GenerationResult]:
        """Execute the command and return results."""
        pass
    
    @abstractmethod
    def get_description(self) -> str:
        """Get a description of what this command does."""
        pass


class BaseCommand(ICommand):
    """Base command with common functionality."""
    
    def __init__(
        self,
        ai_service: IAIService,
        result_repository: IGenerationResultRepository,
        context: ExecutionContext
    ) -> None:
        self.ai_service = ai_service
        self.result_repository = result_repository
        self.context = context
    
    async def _process_page_definition(self, page_def: PageDefinition) -> GenerationResult:
        """Process a single page definition."""
        logger.info(f"Processing page: {page_def.name}")
        
        result = await self.ai_service.generate_code(page_def, self.context)
        await self.result_repository.save_result(result)
        
        return result


class SingleFileGenerationCommand(BaseCommand):
    """Command to generate code for a single page definition."""
    
    def __init__(
        self,
        ai_service: IAIService,
        result_repository: IGenerationResultRepository,
        context: ExecutionContext,
        page_definition: PageDefinition
    ) -> None:
        super().__init__(ai_service, result_repository, context)
        self.page_definition = page_definition
    
    async def execute(self) -> List[GenerationResult]:
        """Execute single file generation."""
        logger.info(f"Generating code for single page: {self.page_definition.name}")
        
        result = await self._process_page_definition(self.page_definition)
        return [result]
    
    def get_description(self) -> str:
        return f"Generate code for single page: {self.page_definition.name}"


class BatchGenerationCommand(BaseCommand):
    """Command to generate code for multiple page definitions."""
    
    def __init__(
        self,
        ai_service: IAIService,
        result_repository: IGenerationResultRepository,
        context: ExecutionContext,
        page_definitions: List[PageDefinition]
    ) -> None:
        super().__init__(ai_service, result_repository, context)
        self.page_definitions = page_definitions
    
    async def execute(self) -> List[GenerationResult]:
        """Execute batch generation."""
        if not self.page_definitions:
            logger.warning("No page definitions found for batch generation")
            return []
        
        logger.info(f"Starting batch generation for {len(self.page_definitions)} pages")
        
        results = []
        
        for i, page_def in enumerate(self.page_definitions, 1):
            try:
                print(f"Processing {i}/{len(self.page_definitions)}: {page_def.name}")
                result = await self._process_page_definition(page_def)
                results.append(result)
                
            except Exception as e:
                logger.error(f"Failed to process {page_def.name}: {e}")
                # Continue with other pages even if one fails
                error_result = GenerationResult(
                    page_definition_id=page_def.id,
                    page_name=page_def.name,
                    status=GenerationStatus.FAILED,
                    error_message=str(e),
                    context=self.context
                )
                results.append(error_result)
                await self.result_repository.save_result(error_result)
        
        successful = len([r for r in results if r.is_successful])
        logger.info(f"Batch generation completed: {successful}/{len(results)} successful")
        return results
    
    def get_description(self) -> str:
        return f"Generate code for {len(self.page_definitions)} pages"


class CommandFactory:
    """Factory for creating command instances."""
    
    def __init__(self, container: ServiceContainer) -> None:
        self.container = container
    
    async def create_command(
        self, 
        mode: ExecutionMode, 
        context: ExecutionContext,
        **kwargs
    ) -> ICommand:
        """Create a command based on execution mode."""
        ai_service = await self.container.get_service(IAIService)
        result_repo = await self.container.get_service(IGenerationResultRepository)
        
        if mode == ExecutionMode.SINGLE_FILE:
            page_definition = kwargs.get('page_definition')
            if not page_definition:
                raise ValueError("page_definition required for single file mode")
            return SingleFileGenerationCommand(ai_service, result_repo, context, page_definition)
        
        elif mode == ExecutionMode.DIRECTORY_BATCH:
            page_definitions = kwargs.get('page_definitions', [])
            return BatchGenerationCommand(ai_service, result_repo, context, page_definitions)
        
        else:
            raise ValueError(f"Unsupported execution mode: {mode}")

## 📄 Output and Presentation Services

Professional output formatting and result presentation:

### **Output Architecture**
- `IOutputService`: Abstract interface for result formatting
- `ConsoleOutputService`: Rich console output with syntax highlighting and statistics

### **Output Features**
- **Summary Statistics**: Success/failure counts and execution times
- **Detailed Results**: Individual page results with generated code
- **Code Presentation**: Line-numbered code with truncation for long outputs
- **Error Reporting**: Clear error messages and exception types
- **Professional Formatting**: Clean, readable output with emojis and sections

### **Key Benefits**
- **Readability**: Well-structured output that's easy to scan and understand
- **Debugging**: Detailed error information for troubleshooting
- **Scalability**: Truncated output prevents overwhelming displays
- **Extensibility**: Easy to add new output formats (JSON, HTML, etc.)

The output service provides professional-grade result presentation suitable for both development and production use.

In [20]:
"""Output and result presentation services."""

class IOutputService(ABC):
    """Abstract interface for output services."""
    
    @abstractmethod
    async def format_results(self, results: List[GenerationResult]) -> str:
        """Format results for display."""
        pass


class ConsoleOutputService(IOutputService):
    """Console-based output service with clean formatting."""
    
    async def format_results(self, results: List[GenerationResult]) -> str:
        """Format results for console display."""
        if not results:
            return "No results to display."
        
        output = []
        output.append("\n" + "=" * 80)
        output.append("🚀 PLAYWRIGHT PAGE OBJECT GENERATION RESULTS")
        output.append("=" * 80)
        
        # Summary statistics
        successful = [r for r in results if r.is_successful]
        failed = [r for r in results if not r.is_successful]
        
        output.append(f"\n📊 SUMMARY:")
        output.append(f"   Total Pages: {len(results)}")
        output.append(f"   ✅ Successful: {len(successful)}")
        output.append(f"   ❌ Failed: {len(failed)}")
        
        if successful:
            avg_time = sum(r.execution_time or 0 for r in successful) / len(successful)
            output.append(f"   ⏱️  Average Time: {avg_time:.2f}s")
        
        output.append("\n" + "-" * 80)
        
        # Detailed results
        for i, result in enumerate(results, 1):
            output.append(f"\n[{i:02d}] 📄 {result.page_name}")
            output.append(f"     Status: {'✅ SUCCESS' if result.is_successful else '❌ FAILED'}")
            
            if result.execution_time:
                output.append(f"     Time: {result.execution_time:.2f}s")
            
            if result.is_successful:
                output.append(f"\n     💻 Generated Code:")
                output.append("     " + "-" * 60)
                
                # Add line numbers to code
                code_lines = result.generated_code.split('\n')
                for line_num, line in enumerate(code_lines[:30], 1):  # Limit to first 30 lines
                    output.append(f"     {line_num:3d} | {line}")
                
                if len(code_lines) > 30:
                    output.append(f"     ... ({len(code_lines) - 30} more lines)")
                
                output.append("     " + "-" * 60)
            else:
                output.append(f"     ❌ Error: {result.error_message}")
                if result.error_type:
                    output.append(f"     🔍 Type: {result.error_type}")
            
            output.append("")
        
        output.append("=" * 80)
        return "\n".join(output)

## 🚀 Main Application Orchestrator

Central coordinator that brings all components together:

### **Application Architecture**
- `PlaywrightTestGenerationApplication`: Main application class coordinating all services
- **Service Registration**: Automatic dependency injection setup
- **Lifecycle Management**: Proper initialization and cleanup
- **Context Manager**: Automatic resource management with async context

### **Key Responsibilities**
- **Configuration Loading**: Environment and override-based configuration
- **Service Wiring**: Dependency injection container setup
- **Execution Coordination**: Command creation and execution
- **Result Presentation**: Output formatting and display
- **Error Handling**: Graceful error management and logging

### **Usage Patterns**
- **Context Manager**: Automatic setup and teardown
- **Async Operations**: Non-blocking execution throughout
- **Configuration Flexibility**: Runtime configuration overrides
- **Error Recovery**: Graceful handling of service failures

### **Production Features**
- **Resource Management**: Proper async disposal of services
- **Logging**: Comprehensive logging throughout the lifecycle
- **Validation**: Configuration and service validation
- **Extensibility**: Easy to extend with new services and features

The application orchestrator provides a clean, professional API for Playwright test generation.

In [21]:
"""Main application orchestrator."""

class PlaywrightTestGenerationApplication:
    """Main application class that coordinates all services."""
    
    def __init__(self) -> None:
        self.container: Optional[ServiceContainer] = None
        self.config: Optional[ApplicationConfig] = None
        self.command_factory: Optional[CommandFactory] = None
        self._initialized = False
    
    async def initialize(self, config_overrides: Optional[Dict[str, Any]] = None) -> None:
        """Initialize the application with configuration."""
        if self._initialized:
            logger.warning("Application already initialized")
            return
        
        try:
            # Load configuration
            config_manager = ConfigurationManager()
            self.config = config_manager.load_configuration(config_overrides)
            
            # Initialize dependency injection container
            self.container = ServiceContainer()
            await self._register_services()
            
            # Initialize command factory
            self.command_factory = CommandFactory(self.container)
            
            self._initialized = True
            logger.info("Application initialized successfully")
            
        except Exception as e:
            logger.error(f"Application initialization failed: {e}")
            raise
    
    async def _register_services(self) -> None:
        """Register all services in the dependency injection container."""
        if not self.config or not self.container:
            raise RuntimeError("Configuration or container not available")
        
        # Register configuration as instance
        self.container.register_instance(ApplicationConfig, self.config)
        
        # Register repositories
        self.container.register_instance(IPageDefinitionRepository, InMemoryPageDefinitionRepository())
        self.container.register_instance(IGenerationResultRepository, InMemoryGenerationResultRepository())
        
        # Register template services
        async def template_service_factory() -> BasePageTemplateService:
            config = await self.container.get_service(ApplicationConfig)
            return BasePageTemplateService(config)
        
        async def prompt_service_factory() -> PromptTemplateService:
            return PromptTemplateService()
        
        self.container.register_factory(BasePageTemplateService, template_service_factory)
        self.container.register_factory(PromptTemplateService, prompt_service_factory)
        
        # Register AI service
        async def ai_service_factory() -> IAIService:
            config = await self.container.get_service(ApplicationConfig)
            template_service = await self.container.get_service(BasePageTemplateService)
            prompt_service = await self.container.get_service(PromptTemplateService)
            
            ai_service = SemanticKernelAIService(config, template_service, prompt_service)
            await ai_service.initialize()
            return ai_service
        
        self.container.register_factory(IAIService, ai_service_factory)
        
        # Register output service
        self.container.register_instance(IOutputService, ConsoleOutputService())
        
        logger.info("All services registered successfully")
    
    async def execute_generation(
        self, 
        mode: ExecutionMode,
        **kwargs
    ) -> List[GenerationResult]:
        """Execute code generation with the specified mode and parameters."""
        if not self._initialized:
            raise RuntimeError("Application not initialized. Call initialize() first.")
        
        # Create execution context
        context = ExecutionContext(
            mode=mode,
            metadata=kwargs
        )
        
        logger.info(f"Starting execution with mode: {mode.value}, session: {context.session_id}")
        
        try:
            # Create and execute command
            command = await self.command_factory.create_command(mode, context, **kwargs)
            logger.info(f"Executing command: {command.get_description()}")
            
            results = await command.execute()
            
            # Display results
            output_service = await self.container.get_service(IOutputService)
            formatted_output = await output_service.format_results(results)
            print(formatted_output)
            
            successful = len([r for r in results if r.is_successful])
            logger.info(f"Execution completed: {successful}/{len(results)} successful")
            return results
            
        except Exception as e:
            logger.error(f"Execution failed: {e}")
            raise
    
    async def shutdown(self) -> None:
        """Gracefully shutdown the application."""
        if not self._initialized:
            return
        
        logger.info("Shutting down application...")
        
        try:
            if self.container:
                await self.container.dispose()
            
            self._initialized = False
            logger.info("Application shutdown completed")
            
        except Exception as e:
            logger.error(f"Error during shutdown: {e}")
    
    @asynccontextmanager
    async def application_context(self, config_overrides: Optional[Dict[str, Any]] = None):
        """Context manager for automatic application lifecycle management."""
        try:
            await self.initialize(config_overrides)
            yield self
        finally:
            await self.shutdown()

## 🎯 Examples and Demonstrations

Ready-to-run examples showcasing the system capabilities:

### **Demo Functions**
- `run_demo()`: Orchestrates all demonstrations with API key validation

### **Example Page Definitions**
- **E-commerce Product Page**: Complex page with multiple tasks (view details, add to cart, wishlist)
- **Login Page**: Standard authentication with error handling
- **Search Page**: Product search with filtering capabilities
- **Checkout Page**: Payment processing workflow

### **Key Features**
- **API Key Validation**: Automatic checking for valid OpenAI credentials
- **Error Handling**: Graceful degradation with helpful error messages
- **Progress Feedback**: Real-time updates during batch processing
- **Professional Output**: Clean, formatted results with statistics

### **Usage Instructions**
1. Set your OpenAI API key: `os.environ['OPENAI_API_KEY'] = 'sk-your-key'`
2. Run the demo cell to see the system in action
3. Examine the generated page object models
4. Modify the examples to test your own page definitions

The demonstrations provide a complete example of using the system for real-world Playwright test generation.

In [22]:
"""Example usage and demonstrations."""

async def demo_single_page_generation():
    """Demonstrate single page generation."""
    
    sample_page_content = """
# E-commerce Product Page

A product details page for an e-commerce website with shopping cart functionality.

## Tasks

### View Product Details
Display comprehensive product information including images, description, and pricing.

#### Steps
1. Load product information
2. Display product images
3. Show product description
4. Display pricing and availability

### Add to Cart
Add the selected product to the shopping cart with quantity selection.

#### Steps
1. Select product quantity
2. Choose product options (size, color, etc.)
3. Click "Add to Cart" button
4. Wait for confirmation message
5. Update cart counter

### Add to Wishlist
Save the product to user's wishlist for later purchase.

#### Steps
1. Click wishlist button
2. Handle login requirement if not authenticated
3. Confirm addition to wishlist
4. Update wishlist indicator
"""
    
    config_overrides = {
        'openai_api_key': os.getenv('OPENAI_API_KEY', 'sk-demo'),
        'max_retries': 2
    }
    
    app = PlaywrightTestGenerationApplication()
    
    try:
        async with app.application_context(config_overrides=config_overrides):
            # Create sample page definition
            page_definition = PageDefinition(
                name="EcommerceProductPage",
                content=sample_page_content
            )
            
            print("🚀 Starting single page generation demo...")
            
            # Execute generation
            results = await app.execute_generation(
                ExecutionMode.SINGLE_FILE,
                page_definition=page_definition
            )
            
            return results
            
    except Exception as e:
        print(f"❌ Demo failed: {e}")
        print("\nPlease ensure:")
        print("1. OPENAI_API_KEY environment variable is set with a valid key")
        print("2. You have internet connectivity")
        print("3. Your OpenAI API key has sufficient credits")
        return []


async def demo_batch_generation():
    """Demonstrate batch generation with multiple sample pages."""
    
    sample_pages = [
        PageDefinition(
            name='LoginPage',
            content="""
# User Login Page

## Tasks

### Authenticate User
Login with username and password

#### Steps
1. Enter username
2. Enter password
3. Click login button
4. Handle success/error states
"""
        ),
        PageDefinition(
            name='SearchPage',
            content="""
# Search Results Page

## Tasks

### Search Products
Search for products with filters

#### Steps
1. Enter search query
2. Apply filters
3. Execute search
4. Display results
"""
        ),
        PageDefinition(
            name='CheckoutPage',
            content="""
# Checkout Process Page

## Tasks

### Complete Purchase
Process payment and complete order

#### Steps
1. Review cart items
2. Enter shipping information
3. Select payment method
4. Confirm and submit order
"""
        )
    ]
    
    config_overrides = {
        'openai_api_key': os.getenv('OPENAI_API_KEY', 'sk-demo')
    }
    
    app = PlaywrightTestGenerationApplication()
    
    try:
        async with app.application_context(config_overrides=config_overrides):
            print("🚀 Starting batch generation demo...")
            
            # Execute batch generation
            results = await app.execute_generation(
                ExecutionMode.DIRECTORY_BATCH,
                page_definitions=sample_pages
            )
            
            return results
            
    except Exception as e:
        print(f"❌ Batch demo failed: {e}")
        return []


async def run_demo():
    """Run demonstration of the Playwright test generation system."""
    
    print("\n" + "=" * 80)
    print("🎯 PLAYWRIGHT TEST GENERATION SYSTEM")
    print("=" * 80)
    print("\nRunning demonstrations...")
    
    # Check API key
    api_key = os.getenv('OPENAI_API_KEY')
    if not api_key or not api_key.startswith('sk-'):
        print("\n❌ OpenAI API key not found or invalid.")
        print("Please set the OPENAI_API_KEY environment variable.")
        print("\nExample:")
        print("import os")
        print("os.environ['OPENAI_API_KEY'] = 'sk-your-api-key-here'")
        return
    
    print("✅ OpenAI API key found")
    
    # Run single page demo
    print("\n" + "="*60)
    print("1️⃣  SINGLE PAGE GENERATION DEMO")
    print("="*60)
    await demo_single_page_generation()
    
    # Run batch demo
    print("\n" + "="*60)
    print("2️⃣  BATCH GENERATION DEMO")
    print("="*60)
    await demo_batch_generation()
    
    print("\n🎉 All demonstrations completed!")


# Execute the demo
print("🚀 Initializing Playwright Test Generation System...")
await run_demo()

2025-07-23 20:34:04,572 - __main__ - INFO - Configuration loaded successfully
2025-07-23 20:34:04,573 - __main__ - INFO - All services registered successfully
2025-07-23 20:34:04,574 - __main__ - INFO - Application initialized successfully
2025-07-23 20:34:04,574 - __main__ - INFO - Starting execution with mode: single_file, session: f3304b94-c784-4f7a-b48e-551451685be7


🚀 Initializing Playwright Test Generation System...

🎯 PLAYWRIGHT TEST GENERATION SYSTEM

Running demonstrations...
✅ OpenAI API key found

1️⃣  SINGLE PAGE GENERATION DEMO
🚀 Starting single page generation demo...


2025-07-23 20:34:04,975 - semantic_kernel.agents.chat_completion.chat_completion_agent - INFO - Both `instructions` (You are a senior Python developer specializing in test automation. Generate clean, production-quality Playwright page objects.) and `prompt_template_config` (## Role
You are an expert Python developer specializing in Playwright test automation.

## Instructions
Generate a high-quality Playwright page object model that inherits from BasePage.

### Requirements
1. Use async/await for all Playwright operations
2. Include comprehensive type hints and docstrings
3. Use descriptive, self-documenting method and variable names
4. Prefer data-testid selectors: [data-testid="element-name"]
5. Handle wait conditions appropriately
6. Follow clean code practices

### Response Format
Respond only with clean Python code without markdown fencing.

## Context
All page objects inherit from this BasePage:
{{$base_page_code}}

## Example
### Input
```
# Login Page
## Tasks
### Login
Authent


🚀 PLAYWRIGHT PAGE OBJECT GENERATION RESULTS

📊 SUMMARY:
   Total Pages: 1
   ✅ Successful: 1
   ❌ Failed: 0
   ⏱️  Average Time: 3.43s

--------------------------------------------------------------------------------

[01] 📄 EcommerceProductPage
     Status: ✅ SUCCESS
     Time: 3.43s

     💻 Generated Code:
     ------------------------------------------------------------
       1 | class ProductPage(BasePage):
       2 |     """Page object for e-commerce product details and interactions."""
       3 | 
       4 |     def __init__(self, page: Page) -> None:
       5 |         """Initialize product page with necessary elements."""
       6 |         super().__init__(page)
       7 |         self.product_images = page.locator('[data-testid="product-images"]')
       8 |         self.product_description = page.locator('[data-testid="product-description"]')
       9 |         self.product_pricing = page.locator('[data-testid="product-pricing"]')
      10 |         self.quantity_input = p

2025-07-23 20:34:08,649 - semantic_kernel.agents.chat_completion.chat_completion_agent - INFO - Both `instructions` (You are a senior Python developer specializing in test automation. Generate clean, production-quality Playwright page objects.) and `prompt_template_config` (## Role
You are an expert Python developer specializing in Playwright test automation.

## Instructions
Generate a high-quality Playwright page object model that inherits from BasePage.

### Requirements
1. Use async/await for all Playwright operations
2. Include comprehensive type hints and docstrings
3. Use descriptive, self-documenting method and variable names
4. Prefer data-testid selectors: [data-testid="element-name"]
5. Handle wait conditions appropriately
6. Follow clean code practices

### Response Format
Respond only with clean Python code without markdown fencing.

## Context
All page objects inherit from this BasePage:
{{$base_page_code}}

## Example
### Input
```
# Login Page
## Tasks
### Login
Authent

...

Processing 2/3: SearchPage


2025-07-23 20:34:14,980 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-07-23 20:34:14,982 - semantic_kernel.connectors.ai.open_ai.services.open_ai_handler - INFO - OpenAI usage: CompletionUsage(completion_tokens=321, prompt_tokens=814, total_tokens=1135, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0))
2025-07-23 20:34:14,983 - __main__ - INFO - Successfully generated code for SearchPage in 2.43s
2025-07-23 20:34:14,983 - __main__ - INFO - Processing page: CheckoutPage


Processing 3/3: CheckoutPage


2025-07-23 20:34:16,850 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-07-23 20:34:16,853 - semantic_kernel.connectors.ai.open_ai.services.open_ai_handler - INFO - OpenAI usage: CompletionUsage(completion_tokens=244, prompt_tokens=818, total_tokens=1062, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0))
2025-07-23 20:34:16,853 - __main__ - INFO - Successfully generated code for CheckoutPage in 1.87s
2025-07-23 20:34:16,854 - __main__ - INFO - Batch generation completed: 3/3 successful
2025-07-23 20:34:16,854 - __main__ - INFO - Execution completed: 3/3 successful
2025-07-23 20:34:16,854 - __main__ - INFO - Shutting down application...
2025-07-23 20:34:16,855 - __main__ - INFO - Application shutdown completed



🚀 PLAYWRIGHT PAGE OBJECT GENERATION RESULTS

📊 SUMMARY:
   Total Pages: 3
   ✅ Successful: 3
   ❌ Failed: 0
   ⏱️  Average Time: 2.73s

--------------------------------------------------------------------------------

[01] 📄 LoginPage
     Status: ✅ SUCCESS
     Time: 3.90s

     💻 Generated Code:
     ------------------------------------------------------------
       1 | from typing import Optional
       2 | from playwright.async_api import Page, Locator
       3 | 
       4 | class UserLoginPage(BasePage):
       5 |     """
       6 |     Page object for user authentication functionality.
       7 |     """
       8 | 
       9 |     def __init__(self, page: Page) -> None:
      10 |         """
      11 |         Initialize the user login page with required elements.
      12 | 
      13 |         Args:
      14 |             page: The Playwright page object for browser interactions
      15 |         """
      16 |         super().__init__(page)
      17 |         self.username

## 🎯 Clean Production-Grade Playwright Test Generation System

This notebook implements a **focused, production-ready** system specifically for Playwright test generation with clean architecture principles.

### 🏗️ **Core Architecture**

1. **Dependency Injection** - Clean service management and configuration
2. **Repository Pattern** - Data access abstraction for page definitions and results
3. **Command Pattern** - Single file vs batch generation strategies
4. **Service Layer** - AI service abstraction with retry logic
5. **Clean Separation** - Each component has a single, clear responsibility

### 🔧 **Key Features**

- **Configuration Management** - Environment-based configuration with validation
- **Error Handling** - Comprehensive exception management with retry logic
- **Type Safety** - Complete type hints with Pydantic validation
- **Clean Output** - Professional formatting of generated code
- **Async Operations** - Non-blocking operations throughout

### 🚀 **Focused on Playwright**

This implementation is specifically designed for:
- **Page Object Model generation** from markdown descriptions
- **Clean, testable code** following Python best practices
- **Playwright-specific patterns** like data-testid selectors
- **Production-quality output** with proper async/await patterns

### 📦 **Ready for Use**

The system can be:
- **Run immediately** with a valid OpenAI API key
- **Extended** with new AI providers by implementing `IAIService`
- **Integrated** into CI/CD pipelines for automated test generation
- **Deployed** as a standalone service or library

### 🔄 **Usage Patterns**

1. **Single Page Generation** - Process individual page definitions
2. **Batch Processing** - Handle multiple pages efficiently
3. **Configuration** - Environment-specific settings
4. **Error Recovery** - Graceful handling of failures

This represents a clean, maintainable implementation focused specifically on Playwright test generation, following the same architectural principles you'd use in enterprise C# development but adapted for Python with modern async patterns.

🎉 **Ready for production use in Playwright test automation pipelines!**